In [21]:
import warnings
from typing import Any, Union, List

import numpy as np
import pandas as pd
from IPython.display import Latex, Markdown, display
from latexifier import latexify

warnings.filterwarnings('ignore')

In [15]:
class matrix(np.ndarray):
    """extends np.ndarray"""

    def __new__(cls, input_array):
        obj = np.asarray(input_array).view(cls)
        return obj

    def inverse_rows(self, i: int, j: int):
        """Swaps i-th and j-th rows of a matrix"""
        self[i], self[j] = self[j].copy(), self[i].copy()

    def inverse_cols(self, i: int, j: int):
        """Swaps i-th and j-th columns of a matrix"""
        self = self.transpose()
        self.inverse_rows(i, j)
        self = self.transpose()


    def norm(self) -> float:
        """Calculates Matrix norm using `np.linalg.norm` with parameter `np.inf`"""
        return np.linalg.norm(self, np.inf)

    def inv(self) -> Any:
        return matrix(np.linalg.inv(self))

In [16]:
def print_tex(*argv) -> None:
    """Displays a LaTex Markdown output"""
    res = ' '.join(['$$'] + [(latexify(arg) if isinstance(arg,
                   np.ndarray) else str(arg)) for arg in argv] + ['$$'])
    display(Markdown(res))


def get_exact_column(col: matrix, n: int) -> str:
    """Prepares a string with LaTex syntax for a column of `col` elements"""
    res = [r"""\begin{pmatrix}""", "\n"]
    for i in range(n):
        line = str(float(col[i][0])) + (r"\\" if i != n-1 else "") + "\n"
        res.append(line)
    res.append(r"""\end{pmatrix}""")
    return ''.join(res)

## Матрица: Вариант 12

In [26]:
n = 3
A = matrix([
    [-1.71126, -0.07170, 1.23615],
    [-0.07170, 1.34019, 0.02903],
    [1.23615, 0.02903, -1.71295]
])
print_tex('A=', A)

$$ A= \begin{pmatrix} -1.711260 & -0.071700 & 1.236150 \\ -0.071700 & 1.340190 & 0.029030 \\ 1.236150 & 0.029030 & -1.712950 \end{pmatrix} $$

## Задание: Вариант 4

Найдем методом Якоби все собственные числа и собственные векторы с точностью $\varepsilon = 0.000001$. Собственные вектора должны иметь единичную длину.

In [47]:
eps = 1.e-6


def jacobi(A: matrix = A, n: int = n, eps: float = eps) -> Union[List[float], List[matrix]]:
    # матрица, столбцы которой будут собственными векторами
    X = matrix(np.identity(n))
    A_ = A.copy()

    def get_i_j(A: matrix = A_, n: int = n) -> Union[int, int]:
        i_, j_ = 0, n - 1
        max_val = np.amin(np.abs(A))
        for i in range(n):
            for j in range(i+1, n):
                if abs(A[i][j]) > max_val:
                    max_val = abs(A[i][j])
                    i_, j_ = i, j
        return i_, j_

    i_, j_ = get_i_j(A_)

    while abs(A_[i_][j_]) > eps:

        d = np.sqrt((A_[i_][i_] - A_[j_][j_])**2 + 4 * A_[i_][j_]**2)

        c = np.sqrt(1 / 2 * (1 + abs(A_[i_][i_] - A_[j_][j_]) / d))

        s = np.sign(A_[i_][j_] * (A_[i_][i_] - A_[j_][j_])) * \
            np.sqrt(1 / 2 * (1 - abs(A_[i_][i_] - A_[j_][j_]) / d))

        A_next = A_.copy()

        for i in range(n):
            for j in range(n):
                if i != i_ and i != j_ and j != i_ and j != j_:
                    continue
                elif i != i_ and i != j_:
                    A_next[i][i_] = c * A_[i][i_] * s * A_[i][j_]
                    A_next[i_][i] = A_next[i][i_]
                elif j != i_ and j != j_:
                    A_next[i][j_] = -s * A_[i][i_] + c * A_[i][j_]
                    A_next[j_][i] = A_next[i][j_]
        A_next[i_][i_] = c**2 * A_[i_][i_] + 2 * \
            c * s * A_[i_][j_] + s**2 * A_[j_][j_]
        A_next[j_][j_] = s**2 * A_[i_][i_] - 2 * \
            c * s * A_[i_][j_] + c**2 * A_[j_][j_]
        A_next[i_][j_] = 0
        A_next[j_][i_] = 0

        A_ = A_next.copy()

        V = matrix(np.zeros((n, n)))
        for i in range(n):
            if i != i_ and i != j_:
                V[i][i] = 1
        V[i_][i_] = c
        V[j_][j_] = c
        V[i_][j_] = -s
        V[j_][i_] = s

        X = X @ V

        i_, j_ = get_i_j(A_)

    return [A_[i][i] for i in range(n)], [matrix(row).reshape(n, 1) for row in X.transpose()]


eigenvals, vecs = jacobi(A, n, eps)
for i in range(n):
    vecs[i] /= vecs[i].norm()
for i in range(n):
    print_tex(f'A x_{i}=', A @ vecs[i], r'\stackrel{?}{=}', eigenvals[i] * vecs[i], f'=\lambda_{i} x_{i}')


$$ A x_0= \begin{pmatrix} -0.475955 \\ -0.042690 \\ -0.475629 \end{pmatrix} \stackrel{?}{=} \begin{pmatrix} -0.475955 \\ 0 \\ -0.475629 \end{pmatrix} =\lambda_0 x_0 $$

$$ A x_1= \begin{pmatrix} -0.057593 \\ 1.340672 \\ 0.014914 \end{pmatrix} \stackrel{?}{=} \begin{pmatrix} -0.006413 \\ 1.340387 \\ 0.006418 \end{pmatrix} =\lambda_1 x_1 $$

$$ A x_2= \begin{pmatrix} 2.946927 \\ 0.087856 \\ -2.948533 \end{pmatrix} \stackrel{?}{=} \begin{pmatrix} 2.946437 \\ 0.028216 \\ -2.948452 \end{pmatrix} =\lambda_2 x_2 $$